In [6]:
import sys, os
sys.path.append(os.path.abspath(".."))     # repo root
import tdse
print(sys.executable)         # should end with .../.venv/bin/python
print(tdse.__file__)          # should point into your repo
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider, Checkbox

# Try to import tdse package
try:
    from tdse import Grid2D, ParaxialPropagator, apply_mask_2d, run2d
    from tdse.units import k0_from_energy
    from tdse.apertures import triangle_mask
except ImportError as e:
    raise ImportError("Cannot import tdse. Run `pip install -e .` from the repo root.") from e


/Users/dmitrijnaumov/Documents/dnaumov_documents/Papers/TwistedBeams_PRD/TDSE-Diffraction/.venv/bin/python
/Users/dmitrijnaumov/Documents/dnaumov_documents/Papers/TwistedBeams_PRD/TDSE-Diffraction/tdse/__init__.py


In [7]:

def make_oam_envelope(x, y, waist_m, ell):
    X, Y = np.meshgrid(x, y, indexing="ij")
    rho2 = X**2 + Y**2
    phi = np.arctan2(Y, X)
    return (np.sqrt(rho2)/waist_m)**abs(ell) * np.exp(-rho2/waist_m**2) * np.exp(1j*ell*phi)


In [8]:

def simulate(E_keV=100.0, ell=5, side_um=0.5, waist_um=1.0,
             FOV_um=8.0, Nx=512, dz_um=5000.0,
             z1_um=20000.0, z2_um=50000.0, z3_um=80000.0,
             auto_view=True, same_axes=False,
             view_frac=0.95, pad=1.2, rotation_deg=0.0):

    # Units
    E_eV = E_keV * 1e3
    k0 = k0_from_energy(E_eV)
    side = side_um * 1e-6
    waist = waist_um * 1e-6
    FOV = FOV_um * 1e-6
    dz = dz_um * 1e-6
    z_list = [z1_um*1e-6, z2_um*1e-6, z3_um*1e-6]

    dx = FOV/Nx
    g = Grid2D(Nx, Nx, dx, dx)
    prop = ParaxialPropagator(g, k0=k0, dz=dz)

    lam = 2*np.pi/k0
    print(f"λ={lam*1e12:.3f} pm | k0={k0:.3e} rad/m | dx={dx*1e9:.2f} nm")

    # mask
    A = triangle_mask(g.x, g.y, center=(0.0,0.0), side=side, rotation_deg=rotation_deg, smooth=10e-9)

    # initial field
    u = make_oam_envelope(g.x, g.y, waist, ell)
    apply_mask_2d(u, A)

    # collect planes
    target_steps = sorted({int(round(z/dz)) for z in z_list})
    max_steps = max(target_steps)+1
    planes = {}

    def on_measure(step, u_xy):
        if step not in target_steps or step in planes:
            return
        I = np.abs(u_xy)**2
        I /= I.max() + 1e-30
        z_here = step*dz
        crop=None
        if auto_view:
            X,Y = np.meshgrid(g.x,g.y,indexing="ij")
            R = np.hypot(X,Y)
            r_flat = R.ravel()
            w_flat = (I/I.sum()).ravel()
            order = np.argsort(r_flat)
            csum = np.cumsum(w_flat[order])
            idx = np.searchsorted(csum, view_frac, side="left")
            r_q = r_flat[order][min(idx,len(order)-1)]
            r_view = pad*max(r_q,3*dx)
            ix = np.where((g.x >= -r_view)&(g.x<=r_view))[0]
            iy = np.where((g.y >= -r_view)&(g.y<=r_view))[0]
            if len(ix) and len(iy):
                crop=(ix[0],ix[-1],iy[0],iy[-1])
        planes[step]=(z_here,I,crop)

    run2d(u,prop,steps=max_steps,on_measure=on_measure)

    # plotting
    shared_extent=None
    if same_axes:
        xmin,xmax=g.x[0],g.x[-1]; ymin,ymax=g.y[0],g.y[-1]
        for _,(_,_,crop) in planes.items():
            if crop:
                i0,i1,j0,j1=crop
                xmin=min(xmin,g.x[i0]); xmax=max(xmax,g.x[i1])
                ymin=min(ymin,g.y[j0]); ymax=max(ymax,g.y[j1])
        shared_extent=[xmin*1e6,xmax*1e6,ymin*1e6,ymax*1e6]

    ncols=len(planes)
    fig,axes=plt.subplots(1,ncols,figsize=(4*ncols,4),constrained_layout=True)
    if ncols==1: axes=[axes]
    for ax,step in zip(axes,sorted(planes.keys())):
        z_here,I,crop=planes[step]
        if auto_view and (crop is not None) and not same_axes:
            i0,i1,j0,j1=crop
            Iplot=I[i0:i1+1,j0:j1+1]
            extent=[g.x[i0]*1e6,g.x[i1]*1e6,g.y[j0]*1e6,g.y[j1]*1e6]
        else:
            Iplot=I
            extent=shared_extent if same_axes else [g.x[0]*1e6,g.x[-1]*1e6,g.y[0]*1e6,g.y[-1]*1e6]
        im=ax.imshow(Iplot.T,origin="lower",extent=extent,aspect="equal",vmin=0,vmax=1)
        ax.set_title(f"z ≈ {z_here*1e6:.0f} µm")
        ax.set_xlabel("x (µm)"); ax.set_ylabel("y (µm)")
        fig.colorbar(im,ax=ax,shrink=0.85,label="norm. intensity")

    plt.suptitle(f"Paraxial triangle | ℓ={ell} | E={E_keV} keV | side={side_um} µm")
    plt.show()


In [9]:

interact(simulate,
         E_keV=FloatSlider(value=100,min=10,max=300,step=10,description="E (keV)"),
         ell=IntSlider(value=5,min=-10,max=10,step=1,description="ℓ"),
         side_um=FloatSlider(value=0.5,min=0.1,max=5,step=0.1,description="side (µm)"),
         waist_um=FloatSlider(value=1.0,min=0.1,max=10,step=0.1,description="waist (µm)"),
         FOV_um=FloatSlider(value=8.0,min=2,max=20,step=1,description="FOV (µm)"),
         Nx=IntSlider(value=512,min=128,max=1024,step=128,description="Nx"),
         dz_um=FloatSlider(value=5000,min=1000,max=20000,step=1000,description="dz (µm)"),
         z1_um=FloatSlider(value=20000,min=1000,max=100000,step=1000,description="z1 (µm)"),
         z2_um=FloatSlider(value=50000,min=1000,max=100000,step=1000,description="z2 (µm)"),
         z3_um=FloatSlider(value=80000,min=1000,max=100000,step=1000,description="z3 (µm)"),
         auto_view=Checkbox(value=True,description="auto_view"),
         same_axes=Checkbox(value=False,description="same_axes"),
         rotation_deg=FloatSlider(value=0.0,min=0.0,max=360,step=0.1,description="rotation of aperture(deg)"));


interactive(children=(FloatSlider(value=100.0, description='E (keV)', max=300.0, min=10.0, step=10.0), IntSlid…